<a href="https://colab.research.google.com/github/analyst-rhie/Dacon/blob/main/%EB%86%8D%EC%B6%95%EC%82%B0%EB%AC%BC%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### train_AT_TSALET_ALL : 학습용 전국 도매시장 거래정보 데이터
 * SALEDATE: 경락 일자
   * 그날 형성된 경락값은 곧 농산물의 기준가격을 뜻한다. 농가가 대형 유통업체와 직거래할 때도 경락값은 가격교섭의 가장 중요한 근거가 된다.　
 * WHSAL_NM: 도매시장
 * CMP_NM: 법인
 * PUM_NM: 품목
 * KIND_NM: 품종
 * DAN_NM: 단위
 * POJ_NM: 포장
 * SIZE_NM: 크기
 * LV_NM: 등급
 * SAN_NM: 산지
 * DANQ: 단위중량
 * QTY: 물량
 * COST: 단가
 * TOT_QTY: 총물량 (음수로 집계된 값은 거래 취소 내역)
 * TOT_AMT: 총금액


 * 주요 참고사항
   * 공휴일이나 일요일같은 경우 도매시장이 열지 않기도 합니다.
     * 0으로 표시된 데이터는 해당 일자에 거래가 없었다고 생각해주시면 됩니다.
     * Score 채점시 정답 가격이 0인 데이터는 제외하고 채점됩니다.
(대회안내 - 규칙 - 평가 - 1차 평가 - 평가산식 코드 참고)
target_idx = np.where(answer!=0)
  * Public 데이터는 Public Score 평가에 사용되는 데이터입니다.
    * Public Score는 Private Score 평가 이전에 본인의 모델 성능을 가늠할 수 있는 점수로, 최종 스코어에는 반영되지 않습니다.
    * Private Score는 최종 스코어에 반영되는 점수이고, 여기에 사용되는 데이터가 Private 데이터입니다.
실제 제출 기간(2021년 9월 28일 ~ 2021년 11월 4일)에 평가되는 것은 Private Score이며,
따라서 실제 제출 기간에 사용하는 데이터는 Private 데이터입니다.
    * Private 데이터는 제출 기간에 농넷 API를 활용해서 얻으실 수 있습니다. (제출 일자 하루 전까지의 최신 정보)
해당 API는 Private 기간 시작 전에 제공 예정입니다.
    * 제출 기간 시작일 이전의 데이터 (2021.08.24 ~ 2021.09.27)는 추후 제공될 API를 활용해서 얻으시면 되고,
이를 Private 데이터로 합쳐서 활용하실 수 있습니다. 다시 말하면, Public 데이터도 모두 활용하여 (2016.01.01~2021.09.27) 기간 내 데이터를 활용하실 수 있으며,
예측일 시점에 얻을 수 있는 모든 데이터는 학습에 활용 가능합니다. (ex 2021년 9월 30일 →  학습셋에 9월 28, 29일 데이터 추가 가능)
    * 가격산출시 거래취소내역(음수로 집계)은 반영하지 않고있습니다.
('코드 공유' -  'AT_TSALET_ALL 파일 전처리 코드' 참고)
(# 날짜별, 품목별, 거래량이 0 이상인 행만 선택)
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]

    * 때문에 train set 생성시 음수인 데이터는 포함이 되지 않고, 이를 학습한 모델 역시 음수값을 예측값으로 내놓지 않을 것입니다.
만약 이와 다르게 전처리를 하셔서 train set에 음수가 포함되어있다면, 추가적으로 음수 데이터 제거 작업을 해주시는 게 좋을 것 같습니다.

    * 참고로 Score 채점시 정답 가격이 0인 데이터는 제외하고 채점됩니다.
(대회안내 - 규칙 - 평가 - 1차 평가 - 평가산식 코드 참고)
target_idx = np.where(answer!=0)
즉, 실제 시장 거래가 없는 일요일, 공휴일 등의 경우는 자동으로 Score 산정 과정에서 제외됩니다.
따라서 제출하실 정답지에 의도적으로 0을 채워넣을 필요는 없습니다.
    * train.csv에서 건고추가 10만원 이상에 거래되는 일자는 2016년 12월 31일, 2020년3월17일 두 개로 파악됩니다. 해당 거래내역에 대해 주최측을 통해 도매시장에 확인해본 결과, 실제로 해당 가격에 거래된 내역이 맞습니다.
하지만 매우 특이한 경우라고 설명해주셨습니다. 따라서 정확한 가격값은 맞으나, 이상치에 해당한다고 생각해주시면 될 것 같습니다. 해당 이상치에 대해서는 적절한 전처리를 해주시면 됩니다.


In [8]:
import pandas as pd
import numpy as np
import lightgbm
from tqdm import tqdm
import warnings

In [9]:
#train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/농산물/public_data/train.csv')
train = pd.read_csv('E:/DACON/235801_2021 농산물 가격예측 AI 경진대회/public_data/train.csv')
train.head(1000)
conn 

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2018-09-22,토요일,684337.5,907.0,1315966.8,981.0,891112.0,551.0,707.2,13937.0,...,46839.4,6720.0,350574.0,1339.0,447468.0,962.0,635244.0,3484.0,19734.0,8456.0
996,2018-09-23,일요일,4831.0,1320.0,40724.0,1058.0,7320.0,719.0,0.0,0.0,...,792.0,3872.0,1879.0,921.0,2859.0,866.0,44600.0,4289.0,503.0,9350.0
997,2018-09-24,월요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,2018-09-25,화요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
!pip install pymysql

In [7]:
import pymysql

#전역변수 선언부
#conn = None
#cur = None
#sql = ""


##전처리
### lag_feature 추가 및 기타 전처리

In [2]:
def preprocessing(temp_df, pum, len_lag) :
    # p_lag, q_lag 추가
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가

    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['month'] = temp_df['date'].dt.month

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0
      n_week = int(week[0])
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue

    # 불필요한 column 제거        
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

In [ ]:
# preprocessing 함수 예시
pum = '배추'
temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
preprocessing(temp_df, pum, len_lag=28)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,p_lag_1,q_lag_1,p_lag_2,q_lag_2,p_lag_3,q_lag_3,p_lag_4,q_lag_4,p_lag_5,q_lag_5,p_lag_6,q_lag_6,p_lag_7,q_lag_7,p_lag_8,q_lag_8,p_lag_9,q_lag_9,p_lag_10,q_lag_10,p_lag_11,q_lag_11,p_lag_12,q_lag_12,p_lag_13,q_lag_13,p_lag_14,q_lag_14,p_lag_15,q_lag_15,p_lag_16,q_lag_16,p_lag_17,q_lag_17,p_lag_18,q_lag_18,p_lag_19,q_lag_19,p_lag_20,q_lag_20,p_lag_21,q_lag_21,p_lag_22,q_lag_22,p_lag_23,q_lag_23,p_lag_24,q_lag_24,p_lag_25,q_lag_25,p_lag_26,q_lag_26,p_lag_27,q_lag_27,p_lag_28,q_lag_28,month,1_week,2_week,4_week
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,420,449,625
1,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,389,454,733
2,329.0,80860.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,0,0,1048
3,0.0,0.0,329.0,80860.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,398,475,638
4,478.0,1422742.5,0.0,0.0,329.0,80860.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,431,511,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.8,1093.0,1279591.6,1133.0,1441152.8,0.0,0.0,1476.0,760499.0,1564.0,763266.0,1561.0,1020033.2,9,0,0,0
1729,1839.0,1856965.0,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.8,1093.0,1279591.6,1133.0,1441152.8,0.0,0.0,1476.0,760499.0,1564.0,763266.0,9,0,0,0
1730,1789.0,1880095.5,1839.0,1856965.0,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.8,1093.0,1279591.6,1133.0,1441152.8,0.0,0.0,1476.0,760499.0,9,0,0,0
1731,1760.0,1661090.9,1789.0,1880095.5,1839.0,1856965.0,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.

### metric 정의

In [ ]:
def nmae(week_answer, week_submission):
    answer = week_answer.to_numpy()
    target_idx = np.where(answer!=0)
    true = answer[target_idx]
    pred = week_submission[target_idx]
    score = np.mean(np.abs(true-pred)/true)
    
    return score


def at_nmae(pred, dataset):
    y_true = dataset.get_label()
    week_1_answer = y_true[0::3]
    week_2_answer = y_true[1::3]
    week_4_answer = y_true[2::3]
    
    week_1_submission = pred[0::3]
    week_2_submission = pred[1::3]
    week_4_submission = pred[2::3]
    
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3
    
    return 'score', score, False

#### 학습정의

In [ ]:
def model_train(x_train, y_train, x_valid, y_valid) :
    params = {'learning_rate': 0.01, 
              'max_depth': 6, 
              'boosting': 'gbdt', 
              'objective': 'regression',  
              'is_training_metric': True, 
              'num_leaves': 100, 
              'feature_fraction': 0.8, 
              'bagging_fraction': 0.8, 
              'bagging_freq': 5, 
              'seed':42,
              'num_threads':8
             }

    model = lightgbm.train(params, 
                   train_set = lightgbm.Dataset(data = x_train, label = y_train),
                   num_boost_round = 10000, 
                   valid_sets = lightgbm.Dataset(data = x_valid, label = y_valid), 
                   init_model = None, 
                   early_stopping_rounds = 100,
                   feval = at_nmae,
                   verbose_eval = False
                    )
    
    return model

### 품목 및 품종별 모델 학습

In [ ]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [ ]:
model_dict = {}
split = 28 #validation

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing(temp_df, pum, len_lag=28)
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0][f'{week_num}_week']
        
        #train, test split
        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = model_train(x_train, y_train, x_valid, y_valid)

  0%|          | 0/21 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

### 추론

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/농산물/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()
# ['2020-09-29', ...]

for date in tqdm(public_date_list) :
    test = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/농산물/public_data/test_files/test_{date}.csv')
    for pum in unique_pum + unique_kind:
        # 예측기준일에 대해 전처리
        temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
        alldata = pd.concat([train, test, temp_test], sort=False).reset_index(drop=True)
        alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
        alldata = alldata.iloc[-28:].reset_index(drop=True)
        alldata = preprocessing(alldata, pum, len_lag=28)
        temp_test = alldata.iloc[-1].astype(float)
        
        # 개별 모델을 활용하여 1,2,4주 후 가격 예측
        for week_num in [1,2,4] :
            temp_model = model_dict[f'{pum}_model_{week_num}']
            result = temp_model.predict(temp_test)
            condition = (submission['예측대상일자']==f'{date}+{week_num}week')
            idx = submission[condition].index
            submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

  0%|          | 0/38 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
100%|██████████| 38/38 [03:54<00:00,  6.17s/it]


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/농산물/baseline2_0920.csv',index=False)